In [35]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

using RigidBodyDynamics
using MeshCatMechanisms
using StaticArrays
using RigidBodyDynamics.Contact
using RigidBodySim

  Activating environment at `D:\Documents\git_workspace\FlyHopper\scripts\Julia\Project.toml`


In [2]:
curdir = pwd()
const URDFPATH = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(URDFPATH, floating=true)
remove_fixed_tree_joints!(bot);

In [30]:
# add environment
rootframe = root_frame(bot)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1))
add_environment_primitive!(bot, ground);

LoadError: syntax: invalid identifier name "?"

In [22]:
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
set_configuration!(state, [1; 0; 0; 0; 0; 0; 0.7])
q = configuration(state)
v = velocity(state)

8-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [36]:
"""
    build_contact_points(model::bot)
    Establish a list of contact points (Point3D). Returns them in a list  
    First is foot point 
    Last 8 are the eight corners that bounds the torso 
"""
function build_contact_points(mech::Mechanism)
    foot = findbody(mech, "Link 2")
    body = findbody(mech, "base_link")
    
    # Foot contact points 
    point = Point3D(default_frame(foot), 0,0,0)
    
    # Body contact points 
    w_b = 0.25/2 # width
    l_b = 0.25/2 # length
    h_b = 0.25/2   # height
    bp1 = Point3D(default_frame(body), -w_b,-l_b, -h_b)
    bp2 = Point3D(default_frame(body), -w_b,-l_b,  h_b)
    bp3 = Point3D(default_frame(body), -w_b, l_b, -h_b)
    bp4 = Point3D(default_frame(body), -w_b, l_b,  h_b)
    bp5 = Point3D(default_frame(body), w_b, -l_b, -h_b)
    bp6 = Point3D(default_frame(body), w_b, -l_b,  h_b)
    bp7 = Point3D(default_frame(body), w_b,  l_b, -h_b)
    bp8 = Point3D(default_frame(body), w_b,  l_b,  h_b)

    return [point, bp1, bp2, bp3, bp4, bp5, bp6, bp7, bp8]
end

build_contact_points

In [44]:
build_contact_points(bot)[1]


Point3D in "after_Joint 2": [0, 0, 0]

In [43]:
add_contact_point!(findbody(bot, "Link 2"), build_contact_points(bot)[1])
contact_points(findbody(bot, "Link 2"))

LoadError: MethodError: no method matching add_contact_point!(::RigidBody{Float64}, ::Point3D{SVector{3, Int64}})
[0mClosest candidates are:
[0m  add_contact_point!(::RigidBody{T}, [91m::DefaultContactPoint{T}[39m) where T at C:\Users\bboks\.julia\packages\RigidBodyDynamics\8B04X\src\rigid_body.jl:173

In [39]:
?add_contact_point!

search: add_contact_point!



```julia
add_contact_point!(body, point)

```

Add a new contact point to the rigid body


In [23]:
center_of_mass(state)

Point3D in "world": [-0.002547131739826878, -3.3470181064453804e-8, 0.697058817292039]

In [24]:
q0, q1 = joints(bot)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    # velocity_range: Return the range of indices into the joint velocity vector v corresponding to joint "joint"
    torques[velocity_range(state, q0)] .= -1 .* velocity(state, q0)
    torques[velocity_range(state, q1)] .= -1 .* velocity(state, q1)
end;

In [25]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

In [26]:
zero_velocity!(state)
# set_configuration!(state, q0, 0.7)
# set_configuration!(state, q1, -0.8);
mvis = MechanismVisualizer(bot, URDFVisuals(URDFPATH));

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8702
└ @ MeshCat C:\Users\bboks\.julia\packages\MeshCat\X2AUA\src\visualizer.jl:73


In [29]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);